In [1]:
import json

from pyspark.sql import SparkSession
from pymongo import MongoClient

In [2]:
client = MongoClient('mongo', 27017)

In [3]:
db = client.test
collection = db.northwind

In [4]:
with open('/workspace/assignments/data/northwind.json') as f:
    file_data = json.load(f)
    result = collection.insert_one(file_data)

In [5]:
client.close()

In [6]:
packages = {
   'org.mongodb.spark:mongo-spark-connector_2.12:10.1.1',
}

In [7]:
spark = (
    SparkSession.builder
    .config('spark.jars.packages', ','.join(packages))
    .config("spark.mongodb.input.uri", "mongodb://mongo:27017/lotr.heroes")
    .config("spark.mongodb.output.uri", "mongodb://mongo:27017/lotr.heroes")
    .config("spark.mongodb.connection.uri", "mongodb://mongo:27017")
    .config("spark.mongodb.database", "lotr")
    .config("spark.mongodb.collection", "heroes")
    .getOrCreate()
)

:: loading settings :: url = jar:file:/home/hduser/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hduser/.ivy2/cache
The jars for the packages stored in: /home/hduser/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-eadc269c-80a9-41a0-9498-648ea32bc829;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;10.1.1 in central
	found org.mongodb#mongodb-driver-sync;4.8.2 in central
	[4.8.2] org.mongodb#mongodb-driver-sync;[4.8.1,4.8.99)
	found org.mongodb#bson;4.8.2 in central
	found org.mongodb#mongodb-driver-core;4.8.2 in central
	found org.mongodb#bson-record-codec;4.8.2 in central
downloading https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.12/10.1.1/mongo-spark-connector_2.12-10.1.1.jar ...
	[SUCCESSFUL ] org.mongodb.spark#mongo-spark-connector_2.12;10.1.1!mongo-spark-connector_2.12.jar (225ms)
downloading https://repo1.maven.org/maven2/org/mongodb/mongodb-driver-sync/4.8.2/mongodb-driver-sync-4.8.2.jar ...
	[SUC

23/04/21 07:25:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [8]:
people = spark.createDataFrame(
    [
        ("Bilbo Baggins", 50),
        ("Gandalf", 1000),
        ("Thorin", 195),
        ("Balin", 178),
        ("Kili", 77),
        ("Dwalin", 169),
        ("Oin", 167),
        ("Gloin", 158),
        ("Fili", 82),
        ("Bombur", None),
    ],
    ["name", "age"],
)


In [9]:
people.write.format("mongodb").mode("overwrite").save()

In [10]:
df = spark.read.format("mongodb").load()
df.printSchema()
df.show()

root
 |-- _id: string (nullable = true)
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)

+--------------------+----+-------------+
|                 _id| age|         name|
+--------------------+----+-------------+
|64423a898897f8054...| 158|        Gloin|
|64423a898897f8054...| 195|       Thorin|
|64423a898897f8054...| 169|       Dwalin|
|64423a898897f8054...| 167|          Oin|
|64423a898897f8054...|1000|      Gandalf|
|64423a898897f8054...| 178|        Balin|
|64423a898897f8054...|null|       Bombur|
|64423a898897f8054...|  77|         Kili|
|64423a898897f8054...|  82|         Fili|
|64423a898897f8054...|  50|Bilbo Baggins|
+--------------------+----+-------------+



In [11]:
df = spark.read.format("mongodb").options(database="test", collection="northwind").load()
df.printSchema()
df.show()

root
 |-- _id: string (nullable = true)
 |-- customers: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- company: string (nullable = true)
 |    |    |-- last_name: string (nullable = true)
 |    |    |-- first_name: string (nullable = true)
 |    |    |-- email_address: string (nullable = true)
 |    |    |-- job_title: string (nullable = true)
 |    |    |-- business_phone: string (nullable = true)
 |    |    |-- home_phone: string (nullable = true)
 |    |    |-- mobile_phone: string (nullable = true)
 |    |    |-- fax_number: string (nullable = true)
 |    |    |-- address: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- state_province: string (nullable = true)
 |    |    |-- zip_postal_code: string (nullable = true)
 |    |    |-- country_region: string (nullable = true)
 |    |    |-- web_page: string (nullable = true)
 |    |    |-- notes: string (nullable = t